This notebook is based on the python B+-tree implementation, that you have to modify as part of Project 4. The implementation in turn tries to mimic the pseudocode described in the book, as faithfully as possible.

### Setup
The code is split across two files: disk_relations.py, and btree.py. The first file contains some basic primitives like Blocks, Relations, Tuples etc., whereas the second one has the B+-Tree implementation (with the caveat that one function does not work).

In [1]:
import math
import sys
sys.path.append("pyfiles")
from disk_relations import *
from btree import *
from create_sample_databases import *

create_sample_databases has a function to create a database with two relations

In [2]:
# Create a sample database
db1 = createDatabase1("univ")

In [3]:
db1.getRelation("instructor").printTuples()

Relation instructor contains 6 blocks
Block No. 0, Type: RelationBlock: ('10101', 'Srinivasan', 'Comp. Sci.', '65000'), ('12121', 'Wu', 'Finance', '90000')
Block No. 1, Type: RelationBlock: ('15151', 'Mozart', 'Music', '40000'), ('22222', 'Einstein', 'Physics', '95000')
Block No. 2, Type: RelationBlock: ('32343', 'El Said', 'History', '60000'), ('33456', 'Gold', 'Physics', '87000')
Block No. 3, Type: RelationBlock: ('45565', 'Katz', 'Comp. Sci.', '75000'), ('58583', 'Califieri', 'History', '62000')
Block No. 4, Type: RelationBlock: ('76543', 'Singh', 'Finance', '80000'), ('76766', 'Crick', 'Biology', '72000')
Block No. 5, Type: RelationBlock: ('83821', 'Brandt', 'Comp. Sci.', '92000'), ('98345', 'Kim', 'Elec. Eng.', '80000')


### B+Trees
`create_sample_databases` also creates several B+-Trees. We can print out
the indexes as follows.

In [4]:
db1.getIndex("instructor", "name").printTree()

Printing BTree Index on Relation instructor on Attribute name
--- Level 0 (root): 
	Block No. 16, Type: BTree, Parent: None: {Block 11}, Mozart, {Block 15}
--- Level 1: 
	Block No. 11, Type: BTree, Parent: 16: {Block 9}, Einstein, {Block 14}, Gold, {Block 12}
	Block No. 15, Type: BTree, Parent: 16: {Block 13}, Srinivasan, {Block 10}
--- Level 2: 
	Block No. 9, Type: BTree, Parent: 11: {Block 5, Tuple 0}, Brandt, {Block 3, Tuple 1}, Califieri, {Block 4, Tuple 1}, Crick, {Block 14}
	Block No. 14, Type: BTree, Parent: 11: {Block 1, Tuple 1}, Einstein, {Block 2, Tuple 0}, El Said, {Block 12}
	Block No. 12, Type: BTree, Parent: 11: {Block 2, Tuple 1}, Gold, {Block 3, Tuple 0}, Katz, {Block 5, Tuple 1}, Kim, {Block 13}
	Block No. 13, Type: BTree, Parent: 15: {Block 1, Tuple 0}, Mozart, {Block 4, Tuple 0}, Singh, {Block 10}
	Block No. 10, Type: BTree, Parent: 15: {Block 0, Tuple 0}, Srinivasan, {Block 0, Tuple 1}, Wu, None


Currently the B+-Tree is printed out level-by-level as shown here. {Block 36} is a pointer to the Block 36, whereas {Block 26, Tuple 0} is a pointer to a specific tuple. 

You can use the DisplayBTree class to draw a proper tree -- this function basically generates an HTML SVG for the B-Tree.

In [5]:
DisplayBTree(db1.getIndex("instructor", "name"))

<btree.DisplayBTree instance at 0x7fa244dec3f8>

Inserting a tuple into the underlying relation will change the B+-Tree. You are encouraged to try this out. Note that: as discussed below, deletes don't fully work.

Let's see what happens when we try to insert 'Gray'.

In [6]:
rel = db1.getRelation("instructor")
rel.insertTuple(Tuple(rel.schema, ('12345', 'Gray', 'Physics', '95000')))
DisplayBTree(db1.getIndex("instructor", "name"))

<btree.DisplayBTree instance at 0x7fa244d7a710>

Now, if we try to insert say 'Davis', we would need to do a split at the second level.

In [7]:
rel.insertTuple(Tuple(rel.schema, ('12346', 'Davis', 'Physics', '95000')))
DisplayBTree(db1.getIndex("instructor", "name"))

<btree.DisplayBTree instance at 0x7fa244d7a200>

### Searching
The following snippet of code does a search using the tree, and prints out the resulting tuples. It also prints out the blocks that were retrieved during the search.

In [8]:
Globals.printBlockAccesses = True
results = db1.getIndex("instructor", "name").searchByRange("M", "S")
if results is not None and len(results) != 0:
    print "Results: " + " ".join([str(ptr.getTuple()) for ptr in results])
else:
    print "No results found"
Globals.printBlockAccesses = False

Retrieving Block No. 16, Type: BTree, Parent: None: {Block 11}, Gold, {Block 20}, Mozart, {Block 15}
Retrieving Block No. 20, Type: BTree, Parent: 16: {Block 12}, Katz, {Block 18}
Retrieving Block No. 18, Type: BTree, Parent: 20: {Block 3, Tuple 0}, Katz, {Block 5, Tuple 1}, Kim, {Block 13}
Retrieving Block No. 13, Type: BTree, Parent: 15: {Block 1, Tuple 0}, Mozart, {Block 4, Tuple 0}, Singh, {Block 10}
Retrieving Block No. 1, Type: RelationBlock: ('15151', 'Mozart', 'Music', '40000'), ('22222', 'Einstein', 'Physics', '95000')
Results: ('15151', 'Mozart', 'Music', '40000')


### Deleting 
The following code finds a tuple and then deletes it from the relation (and effectively the two B+-Trees on the relation). We print out the final trees.

In [9]:
deleteKey = "Srinivasan"
print "Deleting the entry for key " + deleteKey
index = db1.getIndex("instructor", "name")
results = index.searchByKey(deleteKey)
db1.getRelation("instructor").deleteTuple(results[0])
# The BTrees should have been adjusted automatically
#index.printTree()
#db1.getIndex("instructor", "dept_name").printTree()

Deleting the entry for key Srinivasan
Block No. 15, Type: BTree, Parent: 16: {Block 13}, Srinivasan, {Block 10}
** 0 - 13 - 10
** 2 - 10 - 10
Block No. 16, Type: BTree, Parent: None: {Block 11}, Gold, {Block 20}, Mozart, {Block 15}
** 0 - 11 - 15
** 2 - 20 - 15
** 4 - 15 - 15


In [10]:
DisplayBTree(db1.getIndex("instructor", "name"))

<btree.DisplayBTree instance at 0x7fa244de8878>

In [11]:
deleteKey = "Einstein"
print "Deleting the entry for key " + deleteKey
index = db1.getIndex("instructor", "name")
results = index.searchByKey(deleteKey)
db1.getRelation("instructor").deleteTuple(results[0])
DisplayBTree(db1.getIndex("instructor", "name"))

Deleting the entry for key Einstein
Block No. 11, Type: BTree, Parent: 16: {Block 9}, Crick, {Block 19}, Einstein, {Block 14}
** 0 - 9 - 14
** 2 - 19 - 14
** 4 - 14 - 14


<btree.DisplayBTree instance at 0x7fa244dff0e0>

**Task 1**: However, if I try to delete "Brandt", there is an error because the functionality to redistribute is missing (and you have to implement this functionality).

In [12]:
deleteKey = "Brandt"
index = db1.getIndex("instructor", "name")
results = index.searchByKey(deleteKey)
db1.getRelation("instructor").deleteTuple(results[0])
DisplayBTree(db1.getIndex("instructor", "name"))

Block No. 11, Type: BTree, Parent: 16: {Block 9}, Crick, {Block 19}
** 0 - 9 - 9
Redistributing entries between Block No. 9, Type: BTree, Parent: 11: {Block 3, Tuple 1}, Califieri, {Block 19} and Block No. 19, Type: BTree, Parent: 11: {Block 4, Tuple 1}, Crick, {Block 17, Tuple 1}, Davis, {Block 2, Tuple 0}, El Said, {Block 12}


ValueError: Functionality to be implemented

**Important Note**: The tree and the database is in an inconsistent state now, and any more updates will break (until you implement the missing functionality and rerun). If you want to get back to a consistent state, go to "Kernel" menu option, and click "Restart and Clear Output". Then you can run the cells one-by-one again.

In [13]:
DisplayBTree(db1.getIndex("instructor", "name"))

<btree.DisplayBTree instance at 0x7fa244c7dea8>